<a href="https://colab.research.google.com/github/GregB77/calor_run/blob/main/work_on_gpx.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Convert GPX to CSV

In [ ]:
# 1. Installation de la librairie gpxpy (nécessaire car non installée par défaut)
!pip install gpxpy pandas

import gpxpy
import pandas as pd
from google.colab import files
import io

print("--- ÉTAPE 1 : IMPORTATION ---")
print("Veuillez choisir votre fichier .gpx depuis votre ordinateur.")

# 2. Upload du fichier GPX
uploaded = files.upload()

# Récupération du nom du fichier (le premier fichier uploadé)
filename = next(iter(uploaded))

print(f"\nTraitement du fichier : {filename}...")

# 3. Parsing (Lecture) du fichier GPX
data_points = []

with open(filename, 'r') as gpx_file:
    gpx = gpxpy.parse(gpx_file)

    # On parcourt les traces (tracks), segments et points
    for track in gpx.tracks:
        for segment in track.segments:
            for point in segment.points:
                # On ajoute les données dans une liste
                data_points.append({
                    'Latitude': point.latitude,
                    'Longitude': point.longitude,
                    'Altitude': point.elevation,
                    'Temps': point.time
                })

# 4. Création du DataFrame avec Pandas
df = pd.DataFrame(data_points)

# Affichage des 5 premières lignes pour vérification
print("\n--- APERÇU DES DONNÉES ---")
print(df.head())
print(f"\nNombre total de points récupérés : {len(df)}")

# 5. Conversion en CSV et téléchargement
output_filename = filename.replace('.gpx', '.csv')
df.to_csv(output_filename, index=False)

print(f"\n--- TÉLÉCHARGEMENT ---")
print(f"Le fichier '{output_filename}' est en cours de téléchargement...")
files.download(output_filename)